In [1]:
import pandas as pd
import datetime as dt

In [2]:
df = pd.read_csv('cleaned_superstore_for_tableau.csv')
df['Order_Date'] = pd.to_datetime(df['Order_Date'])

In [3]:
snapshot_date = df['Order_Date'].max() + dt.timedelta(days=1)

In [4]:
rfm = df.groupby('Customer_Name').agg({
    'Order_Date': lambda x: (snapshot_date - x.max()).days, # Recency
    'Order_ID': 'count',                                   # Frequency
    'Sales': 'sum'                                         # Monetary
})

In [5]:
rfm.columns = ['Recency', 'Frequency', 'Monetary']

In [6]:
rfm['R_Score'] = pd.qcut(rfm['Recency'], 4, labels=[4, 3, 2, 1]) # Lower recency is better
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 4, labels=[1, 2, 3, 4])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 4, labels=[1, 2, 3, 4])

In [7]:
def assign_segment(row):
    score = int(row['R_Score']) + int(row['F_Score']) + int(row['M_Score'])
    if score >= 9:
        return 'High Value Customer'
    elif score >= 5:
        return 'Mid Value Customer'
    else:
        return 'Low Value Customer'

rfm['Segment'] = rfm.apply(assign_segment, axis=1)

In [8]:
rfm.to_csv('rfm_analysis.csv')

print("Success! 'rfm_analysis.csv' has been created in your folder.")
print(rfm['Segment'].value_counts())

Success! 'rfm_analysis.csv' has been created in your folder.
Segment
Mid Value Customer     408
High Value Customer    295
Low Value Customer      92
Name: count, dtype: int64
